# Tarea 4

### Importante 1: ###
Este notebook está diseñado para ejecutar en Julia. Note que debe incluir (y por lo tanto tener en el mismo directorio) al archivo `choques_utils.jl`.

### Importante 2: ###

Renombre el archivo como: 
**nombre_grupo_tarea_5.ipynb**

**Todo el código generado y la presentación deberá estar incluida en este notebook.**

**El objetivo de la tarea es variado:**

1. Familiarizarse con el fenómeno de la formación de choques.
2. Familiarizarse con los métodos de captura de choques.
3. Transitar por el proceso de implementar un nuevo código.


**1)** En el directorio encontrará dos notebooks con distintos métodos de captura de choque. 
Uno de segundo orden *KT2* o otro de quinto orden, *MP5*. Ambos evolucionan tanto la ecuación de advección, ($u_t = c u_x$), como la de Burgers, $u_t = \frac{1}{2}(u^2)_x$. Lo hacen en un círculo (condiciones de borde periódicas) de longitud L.

Evolucione ambas (con el MP5): 

1. Para la primera constate que la solución se mueve a velocidad constante. 
2. Para la segunda constate que se forma un choque a un dado tiempo. 
    1. Discuta como encontrar cúando se forma el choque y donde.
    2. Vea que el tiempo de formación del choque coincide con el tiempo teórico calculado. 
    3. Observe que el choque decae en el tiempo, encuentre experimentalmente cual es la ley de decaimiento.
    
**2)** Evolucione la ecuación de advección con ambos códigos: 

1. Compare los resultados y describa que sucede. 
2. Compare con lo que obtendría usando un método usual de diferencias finitas de 4to orden. 

Repita el primer punto con la ecuación de Burgers.

**3)** Elija un sistema en D=1 que sea de tipo conservativo e implemente un código para evolucionarlo. 
Estos dos sistemas que siguen se dan como ejemplos, pero ustedes pueden elegir cualquier otro. 
Es mejor si tienen conocimiento del comportamiento del sistema que elijan.

1. Euler: las variables son la densidad $\rho$ y el momento $p^x$, $U=(\rho, p^x)$.
\begin{eqnarray}
\dot{\rho} &=& -\partial_x(p^x) \\
\dot{p^x} &=& -\partial_x(\frac{p^x p^x}{\rho} - p_0 \rho^{\gamma})
\end{eqnarray}
Con un valor recomendado de $\gamma = 1.3$. Debe tomar un dato inicial donde $\rho$ sea positiva y no cercana a cero. La velocidad de propagación del fluido es: $c^2 = \frac{dp}{d\rho} = \gamma \rho p$
    
2. Force-Free: las variables son el campo magnético (componente y) y el vector de Poyting (componentes x e y), $U = (S^x, S^y, B^y)$
\begin{eqnarray}
\dot{S^x} &=& -\partial_x(T^{xx}) = \partial_x(-e + B_x^2) \\
\dot{S^y} &=& -\partial_x(T^{xy}) = \partial_x(B_xB_y) \\
\dot{B^y} &=& \partial_x(v^x B^y + v^y B^x) 
\end{eqnarray}
Donde, $e = \frac{1}{2}(E_z^2 + B_x^2 + B_y^2) = \frac{1}{2}(\frac{S_x^2+S_y^2}{B_x^2+B_y^2}+B_x^2+B_y^2)$ y $v^i = \frac{S^i}{B^2}$. Para esta ecuación $B^x$ es una constante *distinta de cero*. Además $S^i$ tiene que ser perpendicular $\sum_i S^i B^i = 0$ con $\sum_i (v^i) < 1$.



# Métodos conservativos

Resolvemos la ecuación de Burgers (y la ecuación de advección) usando el método KT2 (Kurganov_Tadmor de orden 2) y MP5 (monotonicity preserving 5).

## Grupo 2: NICOLAS ELIAS AMADO, ANA LAURA GARCIA PERCIANTE, NAHUEL JESUS GUTIERREZ, ALMA ROSA MENDEZ

In [ ]:
#using DiffEqOperators
using OrdinaryDiffEq
#using DifferentialEquations

In [ ]:
using Plots

In [ ]:
using Printf

Aquí decidimos la ecuación a resolver. Esto será utilizado a la hora de elegir la función `Flux_x!` más adelante.

In [ ]:
#problem = :advection
problem = :burgers

Incluimos las implementaciones numéricas de los métodos junto con algunas funciones auxiliares

In [ ]:
include("choques_utils.jl") # todas las funciones necesarias

Aquí puede elegir si usar Kurganov-Tadmor o MP5

In [ ]:
KurganovTadmor = false
MonotonicityPreserving5 = false
#KurganovTadmor = true
MonotonicityPreserving5 = true

Definimos la velocidad máxima de propagación. Esto es necesario para ajustar la disipación del método, así como garantizar estabilidad. Lo que importa es tener una cota inferior para la velocidad máxima de propagación de ondas en cualquier punto.

In [ ]:
#Velocidad máxima de propagación

function advectionspeed(U, c)
    return abs(c)
end

function burgersspeed(U, c)
    return maximum(abs, U)  #no encuentro forma de escribir esto sin que aloque memoria...
end

if problem == :advection
    SpeedMax = advectionspeed
elseif problem == :burgers
    SpeedMax = burgersspeed
end

Aquí damos la función ```Flux_x!```. Recuerde que estamos resolviendo problemas del tipo 

$$ u_t = F(u)_x$$

In [ ]:
#Flujos
function advection!(F, U, c)
    @. F = c*U
end

function burgers!(F, U, Fpars)
    @. F = 0.5*U*U
end

if problem == :advection
    Flux_x! = advection!
elseif problem == :burgers
    Flux_x! = burgers!
end

Ahora empezamos a construir finalmente el problema. Primero definimos el intervalo espacial y temporal, junto con el vector donde guardaremos la solución. También elegimos algunos parámetros del problema.

In [ ]:
N = 2000
N_FIELDS = 1

start = 0.0
stop = 2.0*pi
x = range(start, stop =stop, length = N+1)[1:end-1] #de manera que no incluya el último punto
dx = Float64(x.step)  #Δx
h = 1.0/dx
#Inicializamos el dato
u = Array{Float64}(undef, N, N_FIELDS)

du = copy(u)
@. u[:,1] = sin(x)
#Definimos el intervalo de integración y el paso dt
T = 5.00
tspan = (0.0, T)

#CFL = dt/dx
CFL = 0.1
dt = dx * CFL

#Parámetros del problema
if problem == :advection
    eqpars = 1.0
elseif problem == :burgers
    eqpars = false
end

Aquí elegimos el método de resolucion espacial, y preparamos la tupla `par` para integrar más adelante. Varias de estas funciones están definidas en `choques_utils.jl`

In [ ]:
#Elegimos el método que queremos ver
if KurganovTadmor == true
    θ = 2.0  #Este valor tiene que estar entre 1 y 2. Mientras más cerca de 2, menor disipación.
             #Para sistemas de ecuaciones es mejor que esté más cerca de 1 para evitar oscilaciones.
    auxvectors = createKTauxvectors(N_FIELDS)
    scheme! = KT!
    par = (eqpars, h, θ, Flux_x!, SpeedMax, N, N_FIELDS, auxvectors)
    println("Elegido el método KT")
elseif MonotonicityPreserving5 == true
    auxvectors = createMP5auxvectors(N_FIELDS)
    scheme! = mp5!
    par = (eqpars, h, N, N_FIELDS, Flux_x!, SpeedMax, auxvectors)
    println("Elegido el método MP5")
end

La solución es una matriz de tamaño `N*N_Fields`. Para los problemas escalares esto quiere decir que son escencialmente un vector de una única columna, pero puede probar con sistemas de ecuaciones y funcionará igual.
Recuerde que para analizar convergencia es conveniente tomar datos iniciales que sean suaves y que sean polinómicos a trozos.

Ahora estamos en condiciones de definir los problemas, elegir integradores y fijar su comportamiento. Aquí es importante elegir un integrador estable, es decir uno cuya región de estabilidad incluya parte del eje imaginario, y preferentemente que sea TVD (Total Variation Diminishing). En nuestro caso eligiremos el integrador Runge Kutta `SSPRK33`, que es de tercer orden en el tiempo y es TVD.

In [ ]:
prob = ODEProblem(scheme!,u,tspan,par);

In [ ]:
sol = solve(prob,SSPRK33(),dt=dt, saveat = T/100); # Esto es un método TVD
#@benchmark sol = solve(prob,SSPRK33(),dt=dt, saveat = T/100) # Esto es un método TVD

In [ ]:
anim = @animate for t in sol.t
    plt = plot(x, sol(t), ylims = (-1.2,1.2))
    annotate!(plt, 0.0, 1.1, text("T = $(@sprintf("%.2f", t))", :black, :left, 20))
end
gif(anim, "burgers_sin.gif", fps = 30)

### Como voy a estar resolviendo muchas veces mejor hago una función que pueda llamar con el problema, la condición inicial y el método.

In [ ]:
function getsolution(N,T,incond,equ,method)
    if equ==:burgers
        problem = :burgers
    elseif equ==:advection
        problem = :advection
    end
    
    if problem == :advection
    Flux_x! = advection!
    SpeedMax = advectionspeed
    elseif problem == :burgers
    Flux_x! = burgers!
    SpeedMax = burgersspeed
    end

    if method==:MP5
        KurganovTadmor = false
        MonotonicityPreserving5 = true
    elseif method==:KT
        MonotonicityPreserving5 = false
        KurganovTadmor = true
    end
    N_FIELDS = 1

    start = 0.0
    stop = 2.0*pi
    x = range(start, stop =stop, length = N+1)[1:end-1] #de manera que no incluya el último punto
    dx = Float64(x.step)  #Δx
    h = 1.0/dx
    #Inicializamos el dato
    u = Array{Float64}(undef, N, N_FIELDS)
    du = copy(u)
    if incond==:lorentzian
        @. u[:,1] = 2.5*x^4/(1+x^8)
    elseif incond==:triangle
        u.=0.0   
        @. u[(x .> (stop/6.0)) .& (x .< (stop*3/6)),1] .= 0.25(x-stop/6.0)[(x .> (stop/6.0)) .& (x .< (stop*3/6)),1]
    elseif incond==:sinewave
        @. u[:,1] = sin(x)
    elseif incond==:gaussian
        @. u[:,1] = exp(-(x-pi)^2/0.2)
    end
    #Definimos el intervalo de integración y el paso dt
    tspan = (0.0, T)

    #CFL = dt/dx
    CFL = 0.1
    dt = dx * CFL

    #Parámetros del problema
    if problem == :advection
        eqpars = 1.0
    elseif problem == :burgers
        eqpars = false
    end
    if KurganovTadmor == true
        θ = 2.0  #Este valor tiene que estar entre 1 y 2. Mientras más cerca de 2, menor disipación.
                 #Para sistemas de ecuaciones es mejor que esté más cerca de 1 para evitar oscilaciones.
        auxvectors = createKTauxvectors(N_FIELDS)
        scheme! = KT!
        par = (eqpars, h, θ, Flux_x!, SpeedMax, N, N_FIELDS, auxvectors)
        println("Elegido el método KT")
    elseif MonotonicityPreserving5 == true
        auxvectors = createMP5auxvectors(N_FIELDS)
        scheme! = mp5!
        par = (eqpars, h, N, N_FIELDS, Flux_x!, SpeedMax, auxvectors)
        println("Elegido el método MP5")
    end
    
    prob = ODEProblem(scheme!,u,tspan,par);
    sol = solve(prob,SSPRK33(),dt=dt, saveat = T/100);
    return x,sol
end

## Veamos como evoluciona advección el método de Kurganov Tadmor

In [ ]:
@time x,sol = getsolution(500,32*pi,:gaussian,:advection,:KT);

In [ ]:
ktad = plot(xlabel="x",ylabel="u",title="Advección K-T")
ktad2 = plot(xlabel="x",ylabel="u",title="Advección K-T")
plot!(ktad,x, sol(0.00), ylims = (-0.1,1.3),label="sol t=0")
plot!(ktad,x, sol(8pi), ylims = (-0.1,1.3),label="sol t=8pi")
plot!(ktad,x, sol(16pi), ylims = (-0.1,1.3),label="sol t=16pi")
plot!(ktad,x, sol(24pi), ylims = (-0.1,1.3),label="sol t=24pi")
plot!(ktad,x, sol(32pi), ylims = (-0.1,1.3),label="sol t=32pi")
plot!(ktad2,x, sol(0.00),xlim=(pi-0.5,pi+0.5), ylims = (0.8,1.1),label="sol t=0")
plot!(ktad2,x, sol(8pi), xlim=(pi-0.5,pi+0.5),ylims = (0.8,1.1),label="sol t=8pi")
plot!(ktad2,x, sol(16pi),xlim=(pi-0.5,pi+0.5), ylims = (0.8,1.1),label="sol t=16pi")
plot!(ktad2,x, sol(24pi),xlim=(pi-0.5,pi+0.5), ylims = (0.8,1.1),label="sol t=24pi")
plot!(ktad2,x, sol(32pi),xlim=(pi-0.5,pi+0.5), ylims = (0.8,1.1),label="sol t=32pi")
plot(ktad,ktad2,size=(1200,600))

## Veamos como evoluciona advección el método MP5

In [ ]:
@time x,sol = getsolution(500,128*pi,:gaussian,:advection,:MP5);

In [ ]:
mp = plot(xlabel="x",ylabel="u",title="Advección MP5")
mp2 = plot(xlabel="x",ylabel="u",title="Advección MP5")
plot!(mp,x, sol(0.00), ylims = (-0.1,1.3),label="sol t=0")
plot!(mp,x, sol(32pi), ylims = (-0.1,1.3),label="sol t=32pi")
plot!(mp,x, sol(64pi), ylims = (-0.1,1.3),label="sol t=64pi")
plot!(mp,x, sol(96pi), ylims = (-0.1,1.3),label="sol t=96pi")
plot!(mp,x, sol(128pi), ylims = (-0.1,1.3),label="sol t=128pi")
plot!(mp2,x, sol(0.00),xlim=(pi-0.5,pi+0.5), ylims = (0.8,1.1),label="sol t=0")
plot!(mp2,x, sol(32pi), xlim=(pi-0.5,pi+0.5),ylims = (0.8,1.1),label="sol t=32pi")
plot!(mp2,x, sol(64pi),xlim=(pi-0.5,pi+0.5), ylims = (0.8,1.1),label="sol t=64pi")
plot!(mp2,x, sol(96pi),xlim=(pi-0.5,pi+0.5), ylims = (0.8,1.1),label="sol t=96pi")
plot!(mp2,x, sol(128pi),xlim=(pi-0.5,pi+0.5), ylims = (0.8,1.1),label="sol t=128pi")
plot(mp,mp2,size=(1200,600))

## Veamos como evoluciona burgers el método Kurganov Tadmor

In [ ]:
@time x,sol = getsolution(1000,3,:lorentzian,:burgers,:KT);

In [ ]:
anim = @animate for t in sol.t
    plt = plot(x, sol(t), ylims = (-0.1,1.6))
    annotate!(plt, 0.0, 1.5, text("T = $(@sprintf("%.2f", t))", :black, :left, 20))
end
gif(anim, "burgers_KT.gif", fps = 30)

## Veamos como evoluciona burgers el método MP5

In [ ]:
@time x,sol = getsolution(1000,3,:lorentzian,:burgers,:MP5);

In [ ]:
anim = @animate for t in sol.t
    plt = plot(x, sol(t), ylims = (-0.1,1.6))
    annotate!(plt, 0.0, 1.5, text("T = $(@sprintf("%.2f", t))", :black, :left, 20))
end
gif(anim, "burgers_MP5.gif", fps = 30)

## Veamos como decae el Choque

In [ ]:
x,sol = getsolution(2000,5.0,:triangle,:burgers,:MP5);

In [ ]:
h = plot(xlabel="x",ylabel="u")
plot!(h,x, sol(0.00), ylims = (-0.1,0.7),label="sol t=0.0")
plot!(h,x, sol(1.00), ylims = (-0.1,0.7),label="sol t=1.0")
plot!(h,x, sol(2.00), ylims = (-0.1,0.7),label="sol t=2.0")
plot!(h,x, sol(3.00), ylims = (-0.1,0.7),label="sol t=3.0")

## Si comparamos con [la altura predicha](https://www.mathcha.io/editor/MvY7du23ULmHKJMWWJIv137m8cygmK8Yfw3KE2z) obtenemos

In [ ]:
np=30 #funciona mejor si es un multiplo de 3
Heights=[maximum(sol(0.00+(i-1)*3/np)) for i in 1:np]
scatter([(i-1)*3/np for i in 1:np],Heights,label="altura del choque de la solución numérica",xlabel="t",ylabel="h")
L0=2.0/3.0*pi
h0=maximum(sol(0.0))
plot!([(i-1)*3/np for i in 1:np],[1/sqrt((i-1)*(3/np)/(L0*h0)+1/h0^2) for i in 1:np],linewidth=2, label="altura predicha h(t)")

In [ ]:
anim = @animate for t in sol.t
    plt = plot(x, sol(t), ylims = (-0.1,0.6))
    annotate!(plt, 0.0, 0.5, text("T = $(@sprintf("%.2f", t))", :black, :left, 20))
end
gif(anim, "Burgers_Triangle.gif", fps = 30)

## Codifiquemos las Ecuaciones de Euler

In [ ]:
function eulerspeed(U, c)
    gamma=1.3
    P=0.3
    return sqrt(abs(1.3*U[1]*0.3)) + abs(U[2]/U[1])  #no encuentro forma de escribir esto sin que aloque memoria...
end
SpeedMax = eulerspeed
function euler!(F,U, Fpars)
    gamma=1.3
    P=0.3
    F[1] = -U[2]
    F[2] = -U[2]^2/U[1]-P*U[1]^gamma
end
Flux_x! = euler!

In [ ]:
N = 400
N_FIELDS = 2

start = 0.0
stop = 2.0*pi
x = range(start, stop =stop, length = N+1)[1:end-1] #de manera que no incluya el último punto
dx = Float64(x.step)  #Δx
h = 1.0/dx
#Inicializamos el dato
u = Array{Float64}(undef, N, N_FIELDS)

du = copy(u)
@. u[:,1] = 3+sin(x)*1.1
@. u[:,2] = 0.0
#Definimos el intervalo de integración y el paso dt
T = 15.0
tspan = (0.0, T)

#CFL = dt/dx
CFL = 0.1
dt = dx * CFL

eqpars = false

KurganovTadmor = false
MonotonicityPreserving5 = false
KurganovTadmor = true
#MonotonicityPreserving5 = true

if KurganovTadmor == true
    θ = 2.0  #Este valor tiene que estar entre 1 y 2. Mientras más cerca de 2, menor disipación.
             #Para sistemas de ecuaciones es mejor que esté más cerca de 1 para evitar oscilaciones.
    auxvectors = createKTauxvectors(N_FIELDS)
    scheme! = KT!
    par = (eqpars, h, θ, Flux_x!, SpeedMax, N, N_FIELDS, auxvectors)
    println("Elegido el método KT")
elseif MonotonicityPreserving5 == true
    auxvectors = createMP5auxvectors(N_FIELDS)
    scheme! = mp5!
    par = (eqpars, h, N, N_FIELDS, Flux_x!, SpeedMax, auxvectors)
    println("Elegido el método MP5")
end

In [ ]:
prob = ODEProblem(scheme!,u,tspan,par);
@time sol = solve(prob,SSPRK33(),dt=dt, saveat = T/300);

In [ ]:
KurganovTadmor = false
MonotonicityPreserving5 = false
#KurganovTadmor = true
MonotonicityPreserving5 = true

if KurganovTadmor == true
    θ = 2.0  #Este valor tiene que estar entre 1 y 2. Mientras más cerca de 2, menor disipación.
             #Para sistemas de ecuaciones es mejor que esté más cerca de 1 para evitar oscilaciones.
    auxvectors = createKTauxvectors(N_FIELDS)
    scheme! = KT!
    par = (eqpars, h, θ, Flux_x!, SpeedMax, N, N_FIELDS, auxvectors)
    println("Elegido el método KT")
elseif MonotonicityPreserving5 == true
    auxvectors = createMP5auxvectors(N_FIELDS)
    scheme! = mp5!
    par = (eqpars, h, N, N_FIELDS, Flux_x!, SpeedMax, auxvectors)
    println("Elegido el método MP5")
end

In [ ]:
prob2 = ODEProblem(scheme!,u,tspan,par);
@time sol2 = solve(prob,SSPRK33(),dt=dt, saveat = T/300);

In [ ]:
T=15.0
frames=length(sol2.t)
println(frames/T)

In [ ]:
anim = @animate for t in sol2.t
    plt = plot(ylims = (-1.5,7))
    plot!(plt, x, sol(t) , label=["densidad KT" "momento KT"],linewidth=2,linecolor=[:red :maroon])
    plot!(plt, x, sol2(t) , label=["densidad MP5" "momento MP5"],linewidth=2,linecolor=[:black :gray], linestyle=:dash)
    annotate!(plt, 0.0, 6.5, text("T = $(@sprintf("%.1f", t))", :black, :left, 20))
end
gif(anim, "EulerEqs2.gif", fps = 20)